In [ ]:
import pandas as pd
from emutools.tex import StandardTexDoc
import arviz as az
from autumn.infrastructure.remote import springboard
from inputs.constants import SUPPLEMENT_PATH
from aust_covid.calibration import get_targets
import plotly.graph_objects as go
from emutools.calibration import round_sigfig, plot_spaghetti, plot_param_hover_spaghetti
import plotly.express as px

In [ ]:
rts = springboard.task.RemoteTaskStore()
rts.cd('projects/aust_covid/base_case_analysis')
mt = rts.get_managed_task('2023-09-12T2016-check_running_with_spaghetti')
mt.download_all()

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')
targets = get_targets(app_doc)
spaghetti = pd.read_hdf(mt.local.path / 'output' / 'spaghetti.hdf', 's')

In [ ]:
indicators = ['notifications_ma', 'deaths_ma', 'adult_seropos_prop', 'reproduction_number']
plot_spaghetti(spaghetti, indicators, 2, targets)

In [ ]:
idata = az.from_netcdf(mt.local.path / 'output' / 'calibration_out.nc')
plot_param_hover_spaghetti(spaghetti['notifications_ma'], idata)